# Inference of Mask R-CNN with PointRend on the potato images

In [1]:
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import cv2
import random
import csv
from tqdm import tqdm

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog,MetadataCatalog
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

# import PointRend project
from detectron2.projects import point_rend

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 10,10

def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.show()
    
## run on gpu 1
os.environ["CUDA_VISIBLE_DEVICES"]="0"

supported_cv2_formats = (".bmp", ".dib", ".jpeg", ".jpg", ".jpe", ".jp2", ".png", ".pbm", ".pgm", ".ppm", ".sr", ".ras", ".tiff", ".tif")

In [2]:
def check_direxcist(dir):
    if dir is not None:
        if not os.path.exists(dir):
            os.makedirs(dir)  # make new folder

### Initialize some parameters

In [7]:
from utils.mrcnn_tools import bounding_box, visualize_mrcnn

potato_root = "/mnt/data2/PieterBlok/Potato"
training_id = "PointRend_Potato_20230808"
run_id = "field_okadasan_sarabetsu"
rgb_id = "rgb"
class_names = ['Potato']

imagefolder = os.path.join(potato_root, "Images/okadasan_sarabetsu", run_id)
weightsfolder = os.path.join(potato_root, "Weights", training_id)
resultsfolder = os.path.join(potato_root, "Results", run_id)

check_direxcist(weightsfolder)
check_direxcist(resultsfolder)

### Initialize the image inference

In [4]:
cfg = get_cfg()
point_rend.add_pointrend_config(cfg)
cfg.merge_from_file("./detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_X_101_32x8d_FPN_3x_coco.yaml")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.POINT_HEAD.NUM_CLASSES = len(class_names)
cfg.MODEL.WEIGHTS = os.path.join(weightsfolder, "best_model.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set the testing threshold for this model
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.3

model = build_model(cfg)
checkpointer = DetectionCheckpointer(model)
checkpointer.load(cfg.MODEL.WEIGHTS)
predictor = DefaultPredictor(cfg)

[09/06 11:48:14 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                            | Names in Checkpoint                                                                                  | Shapes                                          |
|:----------------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*                         | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| backbone.bottom_up.res2.0.conv2.*                         | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (256,) (256,) (256,) (256,) (256,8,3,3)         |
| backbone.bottom_up.res2.0.conv3.*                  

WARNING [09/06 11:48:14 d2.projects.point_rend.mask_head]: Weight format of PointRend models have changed! Applying automatic conversion now ...
[09/06 11:48:16 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                            | Names in Checkpoint                                                                                  | Shapes                                          |
|:----------------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*                         | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| backbone.bottom_up.res2.0.conv2.*                         | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,

WARNING [09/06 11:48:16 d2.projects.point_rend.mask_head]: Weight format of PointRend models have changed! Applying automatic conversion now ...


### Visualize the output of the algorithm

In [10]:
images = os.listdir(imagefolder)
for image in tqdm(images):
    if image.lower().endswith(supported_cv2_formats) and rgb_id in image:
        img = cv2.imread(os.path.join(imagefolder, image))
        width, height, _ = img.shape
        imgname = os.path.splitext(image)[0]
        
        # Do the image inference and extract the outputs from Mask R-CNN
        outputs = predictor(img)
        instances = outputs["instances"].to("cpu")
        classes = instances.pred_classes.numpy()
        scores = instances.scores.numpy()
        boxes = instances.pred_boxes.tensor.numpy()
        masks = instances.pred_masks.numpy()

        # Visualize the masks
        img_mrcnn = visualize_mrcnn(img, classes, scores, masks, boxes, class_names)
        cv2.imwrite(os.path.join(resultsfolder, image), img_mrcnn)

  0%|                                                   | 0/376 [00:00<?, ?it/s]/home/pieter/anaconda3/envs/peardet/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/pieter/anaconda3/envs/peardet/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/ate